In [1]:
###############
##Version 1:One 
###############

In [2]:
import os
import numpy as np
import six.moves.cPickle as pickle
import torch
import torch.nn as nn
import torch.utils.data as data
import os
import pandas as pd
import torch

# Correct file path
# file_path = '/home/xren451/rxb/phd/Spatial_interpolation/XBSPA/ModIGNNK/data/NDBC/all.npy'
file_path = 'data/NDBC/all.npy'
station_value = np.load(file_path)
station_value=station_value.transpose(2,0,1)
lat_file_path = 'data/NDBC/Station_info_edit.csv'
station_info=np.array(pd.read_csv(lat_file_path,header=None))
station_value = station_value[:, :, 5:13]
station_value=torch.tensor(station_value)

In [3]:
from MOESTKF_functions import Toy_generation, get_complete_stations, Feature_wise_Subgraph
complete_stations, complete_indices = get_complete_stations(station_info, station_value)
K = 5
subgraph_matrix = Feature_wise_Subgraph(station_info, station_value, complete_stations, complete_indices, K)#Subgraph Matrix: (16, 8)

print("Feature-wise Subgraph Matrix:")
print(subgraph_matrix.shape)#Feature-wise Subgraph Matrix: (16, 8)

# Initialize the new matrix with the desired shape
complete_sub_matrix = np.empty((subgraph_matrix.shape[0], subgraph_matrix.shape[1], 6), dtype=object)

# Fill the new matrix
for i, complete_station in enumerate(complete_stations):
    for j in range(subgraph_matrix.shape[1]):
        complete_sub_matrix[i, j] = np.insert(subgraph_matrix[i, j], 0, complete_station)

# Example output for verification
print("Complete_Sub_matrix.shape",complete_sub_matrix.shape)

Feature-wise Subgraph Matrix:
(35, 8)
Complete_Sub_matrix.shape (35, 8, 6)


In [4]:
complete_sub_matrix[:,0,:]

array([['41012', '41008', 'fwyf1', '41013', '42036', '41047'],
       ['41013', '41037', '41038', 'clkn7', '41024', '41008'],
       ['41043', '41046', '42060', '41044', '41052', '42059'],
       ['41047', 'fwyf1', '41049', '41046', '41012', '41048'],
       ['41048', '41049', '41047', '41008', '41013', 'fwyf1'],
       ['41049', '41047', '41048', '41044', '41046', 'fwyf1'],
       ['42002', '42020', '42035', '42001', '42055', 'fwyf1'],
       ['42012', 'dpia1', '42040', '42036', 'cdrf1', '41008'],
       ['42020', 'babt2', '42035', '42002', '42055', '46047'],
       ['42035', '42020', '42040', '42002', '42001', '42012'],
       ['42036', 'cdrf1', '42012', '42040', 'dpia1', '41008'],
       ['42040', '42012', '42036', 'dpia1', '42001', 'cdrf1'],
       ['42055', '42001', '42002', '42020', '42057', '42035'],
       ['42057', '42055', 'mlrf1', '42001', '42058', '41052'],
       ['42058', '42059', '42060', '42057', '41043', '42001'],
       ['44008', '44020', '44025', 'buzm3', '44065', '4

In [5]:
station_value.shape

torch.Size([103, 8784, 8])

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.metrics import mean_squared_error

# Generate station locations
np.random.seed(42)
num_stations = 10
station_coords = np.random.rand(num_stations, 2)

# Temperature data for one year (one data point per hour, 24 hours * 365 days)
time_steps = 24 * 365
temperature_data = np.random.rand(num_stations, time_steps) * 30  # Temperature range 0 to 30 degrees

# Min-Max normalization for each station's temperature series
min_temp = np.min(temperature_data, axis=1).reshape(-1, 1)
max_temp = np.max(temperature_data, axis=1).reshape(-1, 1)
temperature_data_normalized = (temperature_data - min_temp) / (max_temp - min_temp)

# Generate interpolation point location and true temperature values
interpolation_point = np.random.rand(1, 2)
true_interpolation_temp = np.random.rand(time_steps) * 30  # Assume true temperature range is also 0 to 30 degrees

# Normalize true_interpolation_temp
true_min_temp = np.min(true_interpolation_temp)
true_max_temp = np.max(true_interpolation_temp)
true_interpolation_temp_normalized = (true_interpolation_temp - true_min_temp) / (true_max_temp - true_min_temp)

# Define two subgraphs
subgraph_1 = [0, 1, 2, 3, 4]
subgraph_2 = [5, 6, 7, 8, 9]

# Inverse Distance Weighting (IDW) interpolation
def idw_interpolation(station_coords, temperatures, target_point, power=2):
    distances = np.linalg.norm(station_coords - target_point, axis=1)
    weights = 1 / (distances ** power)
    weights /= weights.sum()
    interpolated_value = np.dot(weights, temperatures)
    return interpolated_value

# Apply IDW interpolation to the two subgraphs
idw_estimate_1 = np.array([idw_interpolation(station_coords[subgraph_1], temperature_data_normalized[subgraph_1, t], interpolation_point) for t in range(time_steps)])
idw_estimate_2 = np.array([idw_interpolation(station_coords[subgraph_2], temperature_data_normalized[subgraph_2, t], interpolation_point) for t in range(time_steps)])

# Define GCN model
class GCNModel(nn.Module):
    def __init__(self, num_features, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
    
    def forward(self, x, edge_index, adj):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

def calculate_inverse_distance_adj(station_coords, decay):
    num_nodes = station_coords.shape[0]
    adj = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.linalg.norm(station_coords[i] - station_coords[j])
                adj[i, j] = np.exp(-decay * distance)
    return torch.tensor(adj, dtype=torch.float)

def prepare_data(subgraph, temperature_data, idw_estimate, decay):
    edge_index = torch.tensor([[i, j] for i in range(len(subgraph)) for j in range(len(subgraph))], dtype=torch.long).t().contiguous()
    adj = calculate_inverse_distance_adj(station_coords[subgraph], decay)
    x = torch.tensor(temperature_data[subgraph], dtype=torch.float).t()
    y = torch.tensor(idw_estimate, dtype=torch.float).t()
    data = Data(x=x, edge_index=edge_index, y=y, adj=adj)
    return data

# Initialize models
def initialize_models(hidden_dim):
    model_1 = GCNModel(num_features=len(subgraph_1), hidden_dim=hidden_dim, output_dim=1)
    model_2 = GCNModel(num_features=len(subgraph_2), hidden_dim=hidden_dim, output_dim=1)
    return model_1, model_2

# Define MoE model
class MoEModel(nn.Module):
    def __init__(self, gcn1, gcn2, hidden_dim):
        super(MoEModel, self).__init__()
        self.gcn1 = gcn1
        self.gcn2 = gcn2
        self.gate = nn.Sequential(
            nn.Linear(2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x1, edge_index1, adj1, x2, edge_index2, adj2):
        out1 = self.gcn1(x1, edge_index1, adj1).view(-1, 1)
        out2 = self.gcn2(x2, edge_index2, adj2).view(-1, 1)
        gate_input = torch.cat([out1, out2], dim=1)
        gate_output = self.gate(gate_input)
        out = gate_output[:, 0].unsqueeze(1) * out1 + gate_output[:, 1].unsqueeze(1) * out2
        return out

# Train MoE model
def train_model(moe_model, data_1, data_2, optimizer, criterion, num_epochs=200):
    for epoch in range(num_epochs):
        moe_model.train()
        optimizer.zero_grad()
        
        out = moe_model(data_1.x, data_1.edge_index, data_1.adj, data_2.x, data_2.edge_index, data_2.adj)
        
        loss_1 = criterion(out.squeeze(), data_1.y)
        loss_2 = criterion(out.squeeze(), data_2.y)
        
        total_loss = loss_1 + loss_2
        total_loss.backward()
        
        optimizer.step()
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss_1: {loss_1.item()}, Loss_2: {loss_2.item()}, Total Loss: {total_loss.item()}')

# Predict temperature for the interpolation point
def predict(moe_model, data_1, data_2):
    moe_model.eval()
    with torch.no_grad():
        predicted_temp = moe_model(data_1.x, data_1.edge_index, data_1.adj, data_2.x, data_2.edge_index, data_2.adj)
    return predicted_temp.squeeze().numpy()

# Define loss function and optimizer
criterion = nn.MSELoss()

# MCMC for evaluating decay coefficients
def mcmc_decay_selection(decay_coefficients, num_samples=100):
    samples = []
    current_decay = np.random.choice(decay_coefficients)
    current_mse = evaluate_decay(current_decay)
    samples.append((current_decay, current_mse))

    for _ in range(num_samples):
        proposed_decay = np.random.choice(decay_coefficients)
        proposed_mse = evaluate_decay(proposed_decay)

        acceptance_prob = min(1, np.exp(current_mse - proposed_mse))
        if np.random.rand() < acceptance_prob:
            current_decay = proposed_decay
            current_mse = proposed_mse
        
        samples.append((current_decay, current_mse))
    
    return samples

def evaluate_decay(decay):
    hidden_dim = 16  # Define hidden_dim here
    data_1 = prepare_data(subgraph_1, temperature_data_normalized, idw_estimate_1, decay)
    data_2 = prepare_data(subgraph_2, temperature_data_normalized, idw_estimate_2, decay)
    model_1, model_2 = initialize_models(hidden_dim)
    moe_model = MoEModel(model_1, model_2, hidden_dim)
    optimizer = optim.Adam(moe_model.parameters(), lr=0.01)
    
    train_model(moe_model, data_1, data_2, optimizer, criterion)
    predicted_temp_normalized = predict(moe_model, data_1, data_2)
    mse = mean_squared_error(true_interpolation_temp_normalized, predicted_temp_normalized)
    
    return mse

# Evaluate different decay coefficients using MCMC
decay_coefficients = np.arange(0, 1.6, 0.1)
samples = mcmc_decay_selection(decay_coefficients)
best_sample = min(samples, key=lambda x: x[1])
best_decay = best_sample[0]
best_mse = best_sample[1]

print(f'Best decay coefficient: {best_decay}, MSE: {best_mse}')


Epoch 20/200, Loss_1: 0.028705375269055367, Loss_2: 0.025190144777297974, Total Loss: 0.05389551818370819
Epoch 40/200, Loss_1: 0.015229105949401855, Loss_2: 0.01403720024973154, Total Loss: 0.02926630526781082
Epoch 60/200, Loss_1: 0.01108667254447937, Loss_2: 0.015253136865794659, Total Loss: 0.026339810341596603
Epoch 80/200, Loss_1: 0.012538804672658443, Loss_2: 0.012368292547762394, Total Loss: 0.024907097220420837
Epoch 100/200, Loss_1: 0.012026399374008179, Loss_2: 0.012182282283902168, Total Loss: 0.024208681657910347
Epoch 120/200, Loss_1: 0.012132907286286354, Loss_2: 0.011661911383271217, Total Loss: 0.02379481866955757
Epoch 140/200, Loss_1: 0.011930824257433414, Loss_2: 0.011592824012041092, Total Loss: 0.023523647338151932
Epoch 160/200, Loss_1: 0.011829941533505917, Loss_2: 0.011536753736436367, Total Loss: 0.023366695269942284
Epoch 180/200, Loss_1: 0.011751854792237282, Loss_2: 0.011522194370627403, Total Loss: 0.023274049162864685
Epoch 200/200, Loss_1: 0.011689988896

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.metrics import mean_squared_error
import time

# Generate station locations
np.random.seed(42)
num_stations = 10
station_coords = np.random.rand(num_stations, 2)

# Temperature data for one year (one data point per hour, 24 hours * 365 days)
time_steps = 24 * 365
temperature_data = np.random.rand(num_stations, time_steps) * 30  # Temperature range 0 to 30 degrees

# Min-Max normalization for each station's temperature series
min_temp = np.min(temperature_data, axis=1).reshape(-1, 1)
max_temp = np.max(temperature_data, axis=1).reshape(-1, 1)
temperature_data_normalized = (temperature_data - min_temp) / (max_temp - min_temp)

# Generate interpolation point location and true temperature values
interpolation_point = np.random.rand(1, 2)
true_interpolation_temp = np.random.rand(time_steps) * 30  # Assume true temperature range is also 0 to 30 degrees

# Normalize true_interpolation_temp
true_min_temp = np.min(true_interpolation_temp)
true_max_temp = np.max(true_interpolation_temp)
true_interpolation_temp_normalized = (true_interpolation_temp - true_min_temp) / (true_max_temp - true_min_temp)

# Define two subgraphs
subgraph_1 = [0, 1, 2, 3, 4]
subgraph_2 = [5, 6, 7, 8, 9]

# Inverse Distance Weighting (IDW) interpolation
def idw_interpolation(station_coords, temperatures, target_point, power=2):
    distances = np.linalg.norm(station_coords - target_point, axis=1)
    weights = 1 / (distances ** power)
    weights /= weights.sum()
    interpolated_value = np.dot(weights, temperatures)
    return interpolated_value

# Apply IDW interpolation to the two subgraphs
idw_estimate_1 = np.array([idw_interpolation(station_coords[subgraph_1], temperature_data_normalized[subgraph_1, t], interpolation_point) for t in range(time_steps)])
idw_estimate_2 = np.array([idw_interpolation(station_coords[subgraph_2], temperature_data_normalized[subgraph_2, t], interpolation_point) for t in range(time_steps)])

# Define GCN model
class GCNModel(nn.Module):
    def __init__(self, num_features, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
    
    def forward(self, x, edge_index, adj):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

def calculate_inverse_distance_adj(station_coords, decay):
    num_nodes = station_coords.shape[0]
    adj = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.linalg.norm(station_coords[i] - station_coords[j])
                adj[i, j] = np.exp(-decay * distance)
    return torch.tensor(adj, dtype=torch.float)

def prepare_data(subgraph, temperature_data, idw_estimate, decay):
    edge_index = torch.tensor([[i, j] for i in range(len(subgraph)) for j in range(len(subgraph))], dtype=torch.long).t().contiguous()
    adj = calculate_inverse_distance_adj(station_coords[subgraph], decay)
    x = torch.tensor(temperature_data[subgraph], dtype=torch.float).t()
    y = torch.tensor(idw_estimate, dtype=torch.float).t()
    data = Data(x=x, edge_index=edge_index, y=y, adj=adj)
    return data

# Initialize models
def initialize_models(hidden_dim):
    model_1 = GCNModel(num_features=len(subgraph_1), hidden_dim=hidden_dim, output_dim=1)
    model_2 = GCNModel(num_features=len(subgraph_2), hidden_dim=hidden_dim, output_dim=1)
    return model_1, model_2

# Define MoE model
class MoEModel(nn.Module):
    def __init__(self, gcn1, gcn2, hidden_dim):
        super(MoEModel, self).__init__()
        self.gcn1 = gcn1
        self.gcn2 = gcn2
        self.gate = nn.Sequential(
            nn.Linear(2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x1, edge_index1, adj1, x2, edge_index2, adj2):
        out1 = self.gcn1(x1, edge_index1, adj1).view(-1, 1)
        out2 = self.gcn2(x2, edge_index2, adj2).view(-1, 1)
        gate_input = torch.cat([out1, out2], dim=1)
        gate_output = self.gate(gate_input)
        out = gate_output[:, 0].unsqueeze(1) * out1 + gate_output[:, 1].unsqueeze(1) * out2
        return out

# Train MoE model
def train_model(moe_model, data_1, data_2, optimizer, criterion, num_epochs=200):
    epoch_times = []
    for epoch in range(num_epochs):
        start_time = time.time()
        moe_model.train()
        optimizer.zero_grad()
        
        out = moe_model(data_1.x, data_1.edge_index, data_1.adj, data_2.x, data_2.edge_index, data_2.adj)
        
        loss_1 = criterion(out.squeeze(), data_1.y)
        loss_2 = criterion(out.squeeze(), data_2.y)
        
        total_loss = loss_1 + loss_2
        total_loss.backward()
        
        optimizer.step()
        
        end_time = time.time()
        epoch_time = end_time - start_time
        epoch_times.append(epoch_time)
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss_1: {loss_1.item()}, Loss_2: {loss_2.item()}, Total Loss: {total_loss.item()}, Epoch Time: {epoch_time:.2f} seconds')
    
    return epoch_times

# Predict temperature for the interpolation point
def predict(moe_model, data_1, data_2):
    moe_model.eval()
    with torch.no_grad():
        predicted_temp = moe_model(data_1.x, data_1.edge_index, data_1.adj, data_2.x, data_2.edge_index, data_2.adj)
    return predicted_temp.squeeze().numpy()

# Define loss function and optimizer
criterion = nn.MSELoss()

# MCMC for evaluating decay coefficients
def mcmc_decay_selection(decay_coefficients, num_samples=100):
    samples = []
    current_decay = np.random.choice(decay_coefficients)
    current_mse, _ = evaluate_decay(current_decay)
    samples.append((current_decay, current_mse))
    optimization_times = []

    for _ in range(num_samples):
        start_time = time.time()
        proposed_decay = np.random.choice(decay_coefficients)
        proposed_mse, _ = evaluate_decay(proposed_decay)

        acceptance_prob = min(1, np.exp(current_mse - proposed_mse))
        if np.random.rand() < acceptance_prob:
            current_decay = proposed_decay
            current_mse = proposed_mse
        
        samples.append((current_decay, current_mse))
        end_time = time.time()
        optimization_time = end_time - start_time
        optimization_times.append(optimization_time)
    
    return samples, optimization_times

def evaluate_decay(decay):
    hidden_dim = 16  # Define hidden_dim here
    data_1 = prepare_data(subgraph_1, temperature_data_normalized, idw_estimate_1, decay)
    data_2 = prepare_data(subgraph_2, temperature_data_normalized, idw_estimate_2, decay)
    model_1, model_2 = initialize_models(hidden_dim)
    moe_model = MoEModel(model_1, model_2, hidden_dim)
    optimizer = optim.Adam(moe_model.parameters(), lr=0.01)
    
    epoch_times = train_model(moe_model, data_1, data_2, optimizer, criterion)
    predicted_temp_normalized = predict(moe_model, data_1, data_2)
    mse = mean_squared_error(true_interpolation_temp_normalized, predicted_temp_normalized)
    
    return mse, epoch_times

# Evaluate different decay coefficients using MCMC
decay_coefficients = np.arange(0, 1.6, 0.1)
samples, optimization_times = mcmc_decay_selection(decay_coefficients)
best_sample = min(samples, key=lambda x: x[1])
best_decay = best_sample[0]
best_mse = best_sample[1]

print(f'Best decay coefficient: {best_decay}, MSE: {best_mse}')

# Print the training time per epoch for the best model
best_mse, epoch_times = evaluate_decay(best_decay)
print(f'Training times per epoch for best decay coefficient ({best_decay}):')
for epoch, time_taken in enumerate(epoch_times):
    print(f'Epoch {epoch + 1}: {time_taken:.2f} seconds')

# Print the total optimization time by MCMC
total_optimization_time = sum(optimization_times)
print(f'Total optimization time by MCMC: {total_optimization_time:.2f} seconds')



Epoch 20/200, Loss_1: 0.11969754844903946, Loss_2: 0.07645069807767868, Total Loss: 0.19614824652671814, Epoch Time: 0.00 seconds
Epoch 40/200, Loss_1: 0.05265704542398453, Loss_2: 0.024850834161043167, Total Loss: 0.077507883310318, Epoch Time: 0.01 seconds
Epoch 60/200, Loss_1: 0.04001958668231964, Loss_2: 0.013977598398923874, Total Loss: 0.053997185081243515, Epoch Time: 0.01 seconds
Epoch 80/200, Loss_1: 0.034054722636938095, Loss_2: 0.01148733589798212, Total Loss: 0.04554205760359764, Epoch Time: 0.01 seconds
Epoch 100/200, Loss_1: 0.031161565333604813, Loss_2: 0.010981419123709202, Total Loss: 0.04214298352599144, Epoch Time: 0.01 seconds
Epoch 120/200, Loss_1: 0.02987779676914215, Loss_2: 0.010533077642321587, Total Loss: 0.04041087627410889, Epoch Time: 0.01 seconds
Epoch 140/200, Loss_1: 0.028770051896572113, Loss_2: 0.009956393390893936, Total Loss: 0.03872644528746605, Epoch Time: 0.01 seconds
Epoch 160/200, Loss_1: 0.026229750365018845, Loss_2: 0.00969184935092926, Total 